In [36]:
from pathlib import Path

import pandas as pd
import tensorflow as tf
from PIL import Image
from tensorflow import keras

In [22]:
# Load data
dataset_dir = Path("../Food Datasets/final-dataset")


def load_recipe5k():
    dataset_dir = Path("../Food Datasets/final-dataset")

In [ ]:
# Build model
# def build_model () :

In [40]:
dataset = tf.data.Dataset.from_tensors(([1, 2, 3], "A", ["onion", "green leaves"]))

In [45]:
dataset.element_spec

(TensorSpec(shape=(3,), dtype=tf.int32, name=None),
 TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(2,), dtype=tf.string, name=None))

In [42]:
list(dataset.as_numpy_iterator())

[(array([1, 2, 3]), b'A', array([b'onion', b'green leaves'], dtype=object))]

In [20]:
for x, y, z in dataset:
    print(x)
    print(y.numpy().decode("UTF-8"))
    print(z)

tf.Tensor([1 2 3], shape=(3,), dtype=int32)
A
tf.Tensor([b'onion' b'green leaves'], shape=(2,), dtype=string)


In [37]:
def load_image_to_arr(path):
    image = tf.keras.preprocessing.image.load_img(path)
    return tf.keras.preprocessing.image.img_to_array(image)


def normalize_image(img_tensor):
    return img_tensor / img_tensor.max()


def load_recipe5k_metadata():
    directory = Path("../Food Datasets/final-dataset/metadata/recipes5k_metadata.csv")
    return pd.read_csv(directory, sep="\t")

In [58]:
class OneHotEncoder:
    def __init__(self, all_category_list, all_ingredient_list):
        self.all_food_categories = all_category_list
        self.all_food_categories.sort()
        self.all_food_categories_integer_encoded = (
            self.__encode_categories_to_integers()
        )
        self.all_ingredients = all_ingredient_list
        self.all_ingredients.sort()
        self.all_ingredients_integer_encoded = self.__encode_ingredients_to_integers()

    def get_category_one_hot_encoding(self, category_name):
        index = self.all_food_categories_integer_encoded[category_name]
        assert index != None, f"{category_name} does not have an integer mapping"
        num_classes = len(self.all_food_categories)
        return keras.utils.to_categorical([index], num_classes)

    def get_ingredients_one_hot_encoding(self, ingredient_list):
        ingredient_list = list(
            map(lambda x: self.__transform_ingredient_to_integer, ingredient_list)
        )
        multi_one_hot_layer = tf.keras.layers.CategoryEncoding(
            num_tokens=len(self.all_ingredients), output_mode="multi_hot"
        )
        return multi_one_hot_layer([ingredient_list])

    def __transform_ingredient_to_integer(self, ingredient_name):
        index = self.all_ingredients_integer_encoded[ingredient_name]
        assert index != None, f"{ingredient_name} does not have an integer mapping"
        return index

    def __encode_categories_to_integers(self):
        return {
            category_name: index
            for index, category_name in enumerate(self.all_food_categories)
        }

    def __encode_ingredients_to_integers(self):
        return {
            ingredient_name: index
            for index, ingredient_name in enumerate(self.all_ingredients)
        }

In [28]:
img_arr = load_image_to_arr(
    dataset_dir / "images" / "apple_pie" / "114542501848791040941317630876042494434.jpg"
)

In [31]:
img_arr.shape

(240, 360, 3)

In [32]:
img_arr.max()

255.0

In [33]:
img_arr.min()

0.0

In [35]:
normalize_image(img_arr)

array([[[0.7529412 , 0.77254903, 0.7490196 ],
        [0.7529412 , 0.77254903, 0.7490196 ],
        [0.75686276, 0.7764706 , 0.7529412 ],
        ...,
        [0.41568628, 0.2       , 0.17254902],
        [0.39607844, 0.19607843, 0.18039216],
        [0.3764706 , 0.18431373, 0.16862746]],

       [[0.7490196 , 0.76862746, 0.74509805],
        [0.7529412 , 0.77254903, 0.7490196 ],
        [0.7529412 , 0.77254903, 0.7490196 ],
        ...,
        [0.42352942, 0.21568628, 0.19215687],
        [0.40392157, 0.21176471, 0.19215687],
        [0.37254903, 0.19607843, 0.18431373]],

       [[0.7411765 , 0.7607843 , 0.7372549 ],
        [0.74509805, 0.7647059 , 0.7411765 ],
        [0.74509805, 0.7647059 , 0.7411765 ],
        ...,
        [0.40784314, 0.21568628, 0.1882353 ],
        [0.3882353 , 0.21176471, 0.19215687],
        [0.36862746, 0.19607843, 0.18431373]],

       ...,

       [[0.7411765 , 0.63529414, 0.4509804 ],
        [0.70980394, 0.6       , 0.4117647 ],
        [0.6745098 , 0

In [39]:
load_recipe5k_metadata()

,ID/File Name,Category,Calorie(kcal),Carbohydrate(g),Protein(g),Fat(g),Ingredients
0,211103814294563038966243131439696432980,apple_pie,3.4393,0.2716,0.0161,0.2603,"flour,salt,oil,water,apple,sugar,cinnamon,butter"
1,263538410746730869383121236069019641366,apple_pie,2.8444,0.3591,0.0293,0.1501,"shells,pie,sugar,flour,cinnamon,apple,lemon,bu..."
2,328597835729663550995342992726274622904,apple_pie,3.2812,0.2658,0.0369,0.2419,"apple,lemon,sugar,flour,cinnamon,nut,butter,sa..."
3,192812446104799022112212906789347819248,apple_pie,2.6394,0.3287,0.0245,0.1409,"pie,apple,sugar,corn starch,cinnamon,lemon,but..."
4,191920265948634303435638687483707715644,apple_pie,3.6433,0.3161,0.0464,0.2601,"apple,brown sugar,butter,cinnamon,pepper,nut,pie"
...,...,...,...,...,...,...,...
4821,272335832857708526395823635239664283537,chocolate_ice_cream,2.2738,0.3737,0.0678,0.0955,"sugar,milk,salt,cocoa,egg,chocolate,vanilla"
4822,192293563052674149150721111048588276110,chocolate_ice_cream,2.2090,0.5410,0.0777,0.0513,"milk,sugar,cocoa,vanilla"
4823,132004208048933661182440986468119687667,chocolate_ice_cream,2.8330,0.5621,0.0992,0.0802,"cocoa,egg,sugar,vanilla,oreo cookies"
4824,35227362469635070879297823378326562678,chocolate_ice_cream,1.9474,0.3294,0.0966,0.0728,"sugar,egg,egg,cocoa,milk,vanilla"


AssertionError: 